# 1 - Libraries

In [ ]:
%pip install plot_keras_history

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize, LabelEncoder
from sklearn.utils import class_weight, shuffle
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical

from plot_keras_history import show_history, plot_history

import tensorflow as tf

# For image preparation
import cv2
from skimage.exposure import match_histograms
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from PIL import Image

# os.environ["TF_KERAS"]='1'
#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# 2 - Data collection

In [ ]:
# Charger les données
data = pd.read_csv('/kaggle/input/UBC-OCEAN/train.csv')
data['image_path'] = [''.join(['/kaggle/input/UBC-OCEAN/train_thumbnails/', str(x), '_thumbnail.png']) if ''.join([str(x), '_thumbnail.png']) in os.listdir('/kaggle/input/UBC-OCEAN/train_thumbnails') else ''.join(['/kaggle/input/UBC-OCEAN/train_images/', str(x), '.png']) for x in data['image_id']]

In [ ]:
# Nombre de classes
nb_lab = len(data['label'].unique())

# Liste des classes
le = LabelEncoder()
le.fit_transform(data['label'])
list_lab = le.classes_

In [ ]:
epochs = 15
batch_size = 32

# 3 - Data augmentation

In [ ]:
# Fonction d'augmentation des données
def data_flow_fct(data, datagen, data_type=None, batch_size=None) :

    data_flow = datagen.flow_from_dataframe(data,
                                            #directory=dir_, # Pas besoin
                                            x_col='image_path',  # Utilisez 'image_path' comme colonne des chemins d'images
                                            y_col='label',#_name',
                                            weight_col=None,
                                            target_size=(224, 224),
                                            classes=None,
                                            class_mode='categorical',
                                            batch_size=batch_size,
                                            shuffle=True,
                                            seed=42,
                                            subset=data_type)
    return data_flow

In [ ]:
# Méthode d'augmentation des données
datagen_train = ImageDataGenerator(
#    featurewise_center=True,
#    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.25,# détermine le ration training/validation
    preprocessing_function=preprocess_input)

# 4 - Train/Val split on augmentated data

In [ ]:
# Augmentation des données et split en train et val
train_flow = data_flow_fct(data, datagen_train, data_type='training',batch_size=batch_size)#divisor_train)
val_flow = data_flow_fct(data, datagen_train, data_type='validation',batch_size=batch_size)#divisor_val)

In [ ]:
(len(train_flow)+len(val_flow))*batch_size

# 5 - Image preparation for VGG16

In [ ]:
%%time
# Fonction de préparation des images au format np.array
#3mn36
def image_prep_fct(data):
    prepared_images = []
    prepared_images_np = np.empty((0, 0))
    
    for path in data['image_path']:
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
        
        # Check the number of channels in the image
        if len(img.shape) == 2:
            # If the image has only one channel, replicate it to create a three-channel image
            img = np.stack((img, img, img), axis=-1)

        # Assuming target_size for img_to_array is (224, 224)
        img = img_to_array(img)
        img = img.reshape((img.shape[0], img.shape[1], img.shape[2]))
        #print(img.shape[0], img.shape[1], img.shape[2])
        img = preprocess_input(img)
        prepared_images.append(img)
        prepared_images_np = np.array(prepared_images)
    
    return prepared_images_np

In [ ]:
%%time
#4min
images_np = image_prep_fct(data)
print(images_np.shape)

# 6 - Model training

In [ ]:
# Définition des X et y pour le training et validation et des X_test et y_test pour le test
X = images_np
le = LabelEncoder()
labels_encoded = le.fit_transform(data['label'])
y = to_categorical(labels_encoded)

In [ ]:
# Fonction de création du modèle
def create_model_fct() :
    #weights_path = "/kaggle/input/vgg16-weights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5" # ATTENTION : activer hors connexion
    weights_path = 'imagenet'
    # Charger le modèle VGG16 pré-entraîné
    #base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model0 = VGG16(include_top=False, weights=weights_path, input_shape=(224, 224, 3)) # ATTENTION : activer hors connexion
    
    # Layer non entraînables = on garde les poids du modèle pré-entraîné
    for layer in model0.layers:
        layer.trainable = False

    # Récupérer la sortie de ce réseau
    x = model0.output
    # Compléter le modèle
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(nb_lab, activation='softmax')(x)

    # Définir le nouveau modèle
    model = Model(inputs=model0.input, outputs=predictions)
    # compilation du modèle 
    model.compile(loss="categorical_crossentropy", optimizer='rmsprop', metrics=["accuracy"])

    print(model.summary())
    
    return model

In [ ]:
%%time
# Création du modèle
# 408ms
with tf.device('/gpu:1'): 
    model = create_model_fct()

# Création du callback
model_save_path = "./model_best_weights.h5"
checkpoint = ModelCheckpoint(model_save_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=5)
callbacks_list = [checkpoint, es]

In [ ]:
%%time
# 4min35 for epochs = 1 and batch_size = 32
# Entraîner sur les données d'entraînement
with tf.device('/gpu:0'):
    history = model.fit(train_flow, epochs=epochs, 
                        #steps_per_epoch=len(X_train) // batch_size,
                        steps_per_epoch=len(train_flow),
                        callbacks=callbacks_list, 
                        validation_data=val_flow,
                        #validation_steps=len(X_val) // batch_size,
                        validation_steps=len(val_flow),
                        verbose=1)

# 8 - Performances

In [ ]:
%%time
# Visualisation de l'évolution des performances/epoch
show_history(history)
plot_history(history, path="history.png")
plt.close()

In [ ]:
%%time
# 1min 28 for batch_size = 32
#y_pred = model.predict(val_flow, steps=len(X_val) // batch_size, batch_size=batch_size)
y_pred = model.predict(val_flow, steps=len(val_flow), batch_size=batch_size)

In [ ]:
%%time
# 1min for batch_size = 32
# Nombre total d'échantillons dans le jeu de validation
nombre_total_val = len(val_flow) * batch_size

# Initialisation d'un tableau pour stocker les étiquettes réelles
y_val = np.zeros((nombre_total_val, nb_lab))  

# Itérer sur le générateur pour extraire les étiquettes réelles
for i in range(len(val_flow)):
    _, batch_y_val = val_flow[i]  # Supposons que le générateur génère des paires (X_val, y_val)
    start_index = i * batch_size
    end_index = start_index + len(batch_y_val)
    y_val[start_index:end_index] = batch_y_val

# Maintenant, y_val contient les étiquettes réelles correspondantes aux prédictions

In [ ]:
# Afficher les courbes d'accuracy et de loss
#show_history(history)
plot_history(history)

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, classification_report

# Générer la matrice de confusion
cm = confusion_matrix(y_val.argmax(axis=1)[0:len(y_pred)], y_pred.argmax(axis=1))

# Afficher la matrice de confusion
print("Matrice de confusion :")
print(cm)

# Afficher le rapport de classification
print("\nRapport de classification :")
print(classification_report(y_val.argmax(axis=1)[0:len(y_pred)], y_pred.argmax(axis=1)))

In [ ]:
# Afficher la matrice de confusion
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=list_lab, yticklabels=list_lab, cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()